In [1]:
#path to the data
pathCOMPASOutput = '/Volumes/TOSHIBA EXT/BHNS_all_in/Fiducial/'


In [1]:
#Needed in general
import numpy as np
import matplotlib.pyplot as plt
import sys
import h5py as h5
import gc
#Quick fudge to make import from ../Scripts work
sys.path.append('../../Scripts')

#Custom imports
import ClassFormationChannels as FC 
import coencodeVarious        as CV
import ClassCOMPAS            as CC

# General Idea

96 percent of the BBHs form through three distinct channels.
In this notebook we will go through the steps of the birth of the binaries until the moment they merge as BBHs. We will asses what fractions of all systems drop out at which steps.

In previous git versions you can find a compiled approach
to the fractions in Appendix_fractions. Would be nice experimental code
t formalise. However for clarity I do it explicitly here

We start by gathering the seeds of the three channels and printing them as a reminder


WARNING:

This notebook takes long to run for my simulation of 90million binaries. I used a laptop with 8GB ram and it worked but struggled
a bit. Run at own risk 


# Get the seeds of interest

Based on discussions with Ilya we chose to give the formation channels at a specific metallicity.
We chose tenth of solar.


In [2]:
pathCOMPASOutput='/Volumes/Andromeda/DATA/AllDCO_bugfix/fiducial/'

#But I want only within Hubble time 
Data            = CC.COMPASData(path=pathCOMPASOutput, lazyData=True, Mlower=5., \
                 Mupper=150., binaryFraction=1)
Data.setCOMPASDCOmask(types='BHNS',  withinHubbleTime=True, optimistic=True)
Data.setCOMPASData()
#only want to look at 

IOError: Unable to create file (unable to open file: name = '/Volumes/Andromeda/DATA/AllDCO_bugfix/fiducial/COMPASOutput.h5', errno = 17, error message = 'File exists', flags = 15, o_flags = a02)

In [5]:
#what are the metallicities in terms of solar?
solar   = 0.0142
print np.unique(Data.metallicitySystems)
metallicityInterest = 0.001
#closest metallicity to tenth solar = 0.00132035
#give me the seeds of BBH mergers that have this metallicity
maskZ   = Data.metallicitySystems ==  metallicityInterest
seedsInterest = Data.seeds[maskZ]
print len(seedsInterest)


[0.0001 0.0002 0.0005 0.001  0.002  0.006  0.0142 0.03  ]
32133


# Percentage interacting

In [6]:
#The first step is to look at the number of systems interacting at this metallicity
#at the time of simulating RLOF and CEE printing was not robust yet
# to catch all the systems. Some MS and touching systems might
#not be there but should not be too many

#precentage interacting systems the systems?
#What were all the systems at metallicity Z?
f          = h5.File(pathCOMPASOutput+'COMPASOutput.h5')
maskZ      = f['systems']['Metallicity1'][...].squeeze() == metallicityInterest
allSeedsZ  = f['systems']['SEED'][...].squeeze()[maskZ]
normalisation = float(len(allSeedsZ))
#which seeds are interacting (Z doesnt matter)
RLOFseeds  = np.unique(f['RLOF']['randomSeed'][...].squeeze())
CEEseeds   = np.unique(f['commonEnvelopes']['randomSeed'][...].squeeze())
print RLOFseeds.shape, CEEseeds.shape
InteractingSeeds = np.unique(np.concatenate((RLOFseeds, CEEseeds)))


#Which of the Seeds in SeedsZ are interacting?
boolInteracting  = np.in1d(allSeedsZ, InteractingSeeds)
print 'of all seeds %s percent experiencedMT'\
    %((np.sum(boolInteracting)/normalisation)*100)
#from here on we use the seedsOf interest to do the next step in the formation channel
seedsInterest = allSeedsZ[boolInteracting]    
#these arrays are long delete and clear memory
del maskZ
del allSeedsZ
del CEEseeds
del RLOFseeds
del boolInteracting
del InteractingSeeds
#garbage collect forces deleted arrays to clear from memory
gc.collect()



(4799583,) (3773878,)
of all seeds 70.47153622771017 percent experiencedMT


7

# First moment of mass Transfer

In [7]:
# During this research the RLOF printing was very much in development
# This means that some output such as the x-th moment of RLOF was 
# not robust (yet). Hence we use a clever trick with the seeds
# Credits  idea to J.W.Barrett, made it more general to get not only
# firt RLOF per system but x-th. Snipper is in coencodeVarious.cp

#carefull the cosmicInt simulation is large and this will 
#take a lot of RAM-memory - be patient and hope (fingerscrossed)
#array of 90.000.000 for allSeeds RLOF possibly more for RLOF
#close other windows :O X'(

#UPDATE we only want of metallicity Z
RLOFseeds   = f['RLOF']['randomSeed'][...].squeeze()
RLOF_Z_MASK = np.in1d(RLOFseeds,seedsInterest)

# dont care about 4th and 5th moment etc will take longer to
# calculate those
nthMoment   = CV.getXmomentOfMT(RLOFseeds, maxCounter=3)
del RLOFseeds
gc.collect()

0

In [7]:
print f['commonEnvelopes'].keys()

[u'EccentricityPostCEE', u'EccentricityPreCEE', u'SemiMajorAxisPostCEE', u'SemiMajorAxisPreCEE', u'alphaCE', u'bEFixed1', u'bEFixed2', u'bEKruckow1', u'bEKruckow2', u'bELoveridge1', u'bELoveridge2', u'bELoveridgeWinds1', u'bELoveridgeWinds2', u'bENanjing1', u'bENanjing2', u'bindingEnergy1', u'bindingEnergy2', u'counterCEE', u'finalType1', u'finalType2', u'lambda1', u'lambda2', u'lambdaFixed1', u'lambdaFixed2', u'lambdaKruckow1', u'lambdaKruckow2', u'lambdaLoveridge1', u'lambdaLoveridge2', u'lambdaLoveridgeWinds1', u'lambdaLoveridgeWinds2', u'lambdaNanjing1', u'lambdaNanjing2', u'mass1', u'mass1ZAMS', u'mass2', u'mass2ZAMS', u'massCore1', u'massCore2', u'massEnvelope1', u'massEnvelope2', u'massTransferTrackerHistory', u'optimisticCommonEnvelopeFlag', u'radius1', u'radius1AfterStripping', u'radius2', u'radius2AfterStripping', u'randomSeed', u'rocheLobe1to2PostCEE', u'rocheLobe1to2PreCEE', u'rocheLobe2to1PostCEE', u'rocheLobe2to1PreCEE', u'stellarMerger', u'time', u'type1', u'type2']


# Classic Channel 

In [8]:
# percentage of systems post MS primaries
# MS transfering onto MS secondary  (channel 1 and 2)
donorPostMS = ((f['RLOF']['type1Prev'][...].squeeze() >1) &\
              (f['RLOF']['type1Prev'][...].squeeze() <7))[RLOF_Z_MASK]
companionMS = ((f['RLOF']['type2Prev'][...].squeeze() == 1) |\
              (f['RLOF']['type2Prev'][...].squeeze() == 0))[RLOF_Z_MASK]
primary     = (f['RLOF']['flagRLOF1'][...].squeeze() == True)[RLOF_Z_MASK]
firstMoment = (nthMoment == 1)[RLOF_Z_MASK]

maskInterest      = [donorPostMS & companionMS & firstMoment & primary]
seedsRemain       = f['RLOF']['randomSeed'][...].squeeze()[RLOF_Z_MASK][maskInterest]
nrSystemsInterest = len(seedsRemain)
print "post MS primar onto MS secondary "
print "of interacting systems %s percentage"\
      %((nrSystemsInterest/normalisation)*100)
del donorPostMS
del companionMS
del firstMoment
del primary
del maskInterest
gc.collect()


post MS primar onto MS secondary 
of interacting systems 46.968624251 percentage


/Users/floorbroekgaarden/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if __name__ == '__main__':


0

In [9]:
# test the number of stable mass transfers.
donorPostMS = ((f['commonEnvelopes']['type1'][...].squeeze() >1) &\
              (f['commonEnvelopes']['type1'][...].squeeze() <7))
companionMS = (f['commonEnvelopes']['type2'][...].squeeze() == 1) |\
              (f['commonEnvelopes']['type2'][...].squeeze() == 0)
seeds        = np.in1d(f['commonEnvelopes']['randomSeed'][...].squeeze(), seedsRemain)
maskInterest = donorPostMS & companionMS & seeds
seedsCEE    = f['commonEnvelopes']['randomSeed'][...].squeeze()\
                                [maskInterest]
seedsRemain = seedsRemain[np.logical_not(np.in1d(seedsRemain, seedsCEE))]
nrStable     = float(len(seedsRemain))
print "percentage stable after 1st MT = %s"  %((nrStable/normalisation)*100)
del seedsCEE
del donorPostMS
del companionMS
del maskInterest
gc.collect()

KeyError: "Unable to open object (object 'type1' doesn't exist)"

In [10]:
# nr systems that survived the first SN
primary   = f['supernovae']['whichStar'][...].squeeze() ==1
survived  = f['supernovae']['Survived'][...].squeeze() == True
BH        = f['supernovae']['RemnantType'][...].squeeze() == 14
maskInterest = BH & primary & survived
print np.sum(maskInterest)
seedsSN   = f['supernovae']['randomSeed'][...].squeeze()[maskInterest]
seedsRemain = seedsRemain[(np.in1d(seedsRemain, seedsSN))]
print "percentage surviving first SN = %s"  %((len(seedsRemain)/normalisation)*100)
del primary
del BH
del survived
del seedsSN
gc.collect()

1018277
percentage surviving first SN = 11.6828065375


0

In [11]:
#nr systems second mass transfer 
secondMoment = (nthMoment == 2)[RLOF_Z_MASK]
secondary    = (f['RLOF']['flagRLOF2'][...].squeeze() == True)[RLOF_Z_MASK]
seeds        = np.in1d(f['RLOF']['randomSeed'][...].squeeze()[RLOF_Z_MASK], seedsRemain)
maskInterest = secondMoment & secondary & seeds
seedsRemain  = f['RLOF']['randomSeed'][...].squeeze()[RLOF_Z_MASK][maskInterest]
print "percentage having a second MT from secondary  = %s"  %((len(seedsRemain)/normalisation)*100)
del secondMoment
del secondary
del seeds
gc.collect()
print seedsRemain[0]

percentage having a second MT from secondary  = 10.4825161399
4259793261020842


In [12]:
# test the number of stable mass transfers.
donorUnstripped = (f['commonEnvelopes']['type2'][...].squeeze() <7)
#donor not allowed to be a HG star
seeds           = np.in1d(f['commonEnvelopes']['randomSeed'][...].squeeze(), seedsRemain)
maskInterest    = seeds & donorUnstripped 
seedsRemain        = f['commonEnvelopes']['randomSeed'][...].squeeze()[maskInterest]
print "percentage unstable second MT =  %s"  %((len(seedsRemain)/normalisation)*100)

KeyError: "Unable to open object (object 'type2' doesn't exist)"

In [13]:
survived        = (f['commonEnvelopes']['stellarMerger'][...].squeeze() == False)
#we do the pessimistic assumption
pessimistic     = f['commonEnvelopes']['optimisticCommonEnvelopeFlag'][...].squeeze() ==  False
#they should also not RLOF after stripping directly
R1              = f['commonEnvelopes']['radius1AfterStripping'][...].squeeze()
RLOF1           = f['commonEnvelopes']['rocheLobe1to2PostCEE'][...].squeeze()
R2              = f['commonEnvelopes']['radius2AfterStripping'][...].squeeze()
RLOF2           = f['commonEnvelopes']['rocheLobe2to1PostCEE'][...].squeeze()

noRLOF          = (np.divide(R1, RLOF1) < 1) & (np.divide(R2, RLOF2) < 1)

seeds           = np.in1d(f['commonEnvelopes']['randomSeed'][...].squeeze(), seedsRemain)
maskInterest    = seeds &  survived & pessimistic & noRLOF
seedsRemain        = f['commonEnvelopes']['randomSeed'][...].squeeze()[maskInterest]
print "percentage survived =  %s"  %((len(seedsRemain)/normalisation)*100)
del survived
gc.collect()

/home/cneijssel/.local/lib/python2.7/site-packages/ipykernel/__main__.py:10: RuntimeWarning: invalid value encountered in less


percentage survived =  0.496133333333


0

In [14]:
# nr systems that survived the second SN
secondary    = f['supernovae']['whichStar'][...].squeeze() ==2
survived     = f['supernovae']['Survived'][...].squeeze() == True
BH           = f['supernovae']['RemnantType'][...].squeeze() == 14
seeds        = np.in1d(f['supernovae']['randomSeed'][...].squeeze(), seedsRemain)
maskInterest = BH & secondary & survived & seeds
seedsRemain  = f['supernovae']['randomSeed'][...].squeeze()[maskInterest]
print "percentage surviving second SN = %s"  %((len(seedsRemain)/normalisation)*100)
del secondary
del BH
del survived
gc.collect()

percentage surviving second SN = 0.427


0

In [15]:
seeds        = np.in1d(f['doubleCompactObjects']['seed'][...].squeeze(), seedsRemain)
Hubble       = f['doubleCompactObjects']['mergesInHubbleTimeFlag'][...].squeeze() == True
#to double check R1 RLOF1 method before I double check effect of CEE2 mask (there is none so good :) )
CEE2         = f['doubleCompactObjects']['RLOFSecondaryAfterCEE'][...].squeeze() == False
maskInterest = seeds & Hubble & CEE2
seedsRemain  = f['doubleCompactObjects']['seed'][...].squeeze()[maskInterest]
print np.sum(maskInterest)
print "percentage merging within Hubble time = %s"  %((len(seedsRemain)/normalisation)*100)

del CEE2
del Hubble 
gc.collect()

7193
percentage merging within Hubble time = 0.239766666667


0

# Non CEE Channel

I go through the same steps except making the 2nd MT stable 


In [29]:
# percentage of systems post MS primaries
# MS transfering onto MS secondary  (channel 1 and 2)
donorPostMS = ((f['RLOF']['type1Prev'][...].squeeze() >1) &\
              (f['RLOF']['type1Prev'][...].squeeze() <7))[RLOF_Z_MASK]
companionMS = ((f['RLOF']['type2Prev'][...].squeeze() == 1) |\
              (f['RLOF']['type2Prev'][...].squeeze() == 0))[RLOF_Z_MASK]
primary     = (f['RLOF']['flagRLOF1'][...].squeeze() == True)[RLOF_Z_MASK]
firstMoment = (nthMoment == 1)[RLOF_Z_MASK]

maskInterest      = [donorPostMS & companionMS & firstMoment & primary]
seedsRemain       = f['RLOF']['randomSeed'][...].squeeze()[RLOF_Z_MASK][maskInterest]
nrSystemsInterest = len(seedsRemain)
print "post MS primar onto MS secondary "
print "of interacting systems %s percentage"\
      %((nrSystemsInterest/normalisation)*100)
del donorPostMS
del companionMS
del firstMoment
del primary
del maskInterest
gc.collect()


post MS primar onto MS secondary 
of interacting systems 49.2590666667 percentage


0

In [30]:
# test the number of stable mass transfers.
donorPostMS = ((f['commonEnvelopes']['type1'][...].squeeze() >1) &\
              (f['commonEnvelopes']['type1'][...].squeeze() <7))
companionMS = (f['commonEnvelopes']['type2'][...].squeeze() == 1) |\
              (f['commonEnvelopes']['type2'][...].squeeze() == 0)
seeds        = np.in1d(f['commonEnvelopes']['randomSeed'][...].squeeze(), seedsRemain)
maskInterest = donorPostMS & companionMS & seeds
seedsCEE    = f['commonEnvelopes']['randomSeed'][...].squeeze()\
                                [maskInterest]
seedsRemain = seedsRemain[np.logical_not(np.in1d(seedsRemain, seedsCEE))]
nrStable     = float(len(seedsRemain))
print "percentage stable after 1st MT = %s"  %((nrStable/normalisation)*100)
del seedsCEE
del donorPostMS
del companionMS
del maskInterest
gc.collect()

percentage stable after 1st MT = 23.0556333333


0

In [31]:
# nr systems that survived the first SN
primary   = f['supernovae']['whichStar'][...].squeeze() ==1
survived  = f['supernovae']['Survived'][...].squeeze() == True
BH        = f['supernovae']['RemnantType'][...].squeeze() == 14
maskInterest = BH & primary & survived
print np.sum(maskInterest)
seedsSN   = f['supernovae']['randomSeed'][...].squeeze()[maskInterest]
seedsRemain = seedsRemain[(np.in1d(seedsRemain, seedsSN))]
print "percentage surviving first SN = %s"  %((len(seedsRemain)/normalisation)*100)
del primary
del BH
del survived
del seedsSN
gc.collect()

4320585
percentage surviving first SN = 2.6639


0

In [32]:
#nr systems second mass transfer 
secondMoment = (nthMoment == 2)[RLOF_Z_MASK]
secondary    = (f['RLOF']['flagRLOF2'][...].squeeze() == True)[RLOF_Z_MASK]
seeds        = np.in1d(f['RLOF']['randomSeed'][...].squeeze()[RLOF_Z_MASK], seedsRemain)
maskInterest = secondMoment & secondary & seeds
seedsRemain  = f['RLOF']['randomSeed'][...].squeeze()[RLOF_Z_MASK][maskInterest]
print "percentage having a second MT from secondary  = %s"  %((len(seedsRemain)/normalisation)*100)
del secondMoment
del secondary
del seeds
gc.collect()
print seedsRemain[0]

percentage having a second MT from secondary  = 2.22736666667
8204607777667079584


In [33]:
# test the number of stable mass transfers.
donorUnstripped = (f['commonEnvelopes']['type2'][...].squeeze() <7)
#donor not allowed to be a HG star
seeds           = np.in1d(f['commonEnvelopes']['randomSeed'][...].squeeze(), seedsRemain)
maskInterest    = seeds & donorUnstripped 
seedsCEE    = f['commonEnvelopes']['randomSeed'][...].squeeze()\
                                [maskInterest]
seedsRemain = seedsRemain[np.logical_not(np.in1d(seedsRemain, seedsCEE))]
print "percentage stable second MT =  %s"  %((len(seedsRemain)/normalisation)*100)

percentage stable second MT =  1.35356666667


In [34]:
# nr systems that survived the second SN
secondary    = f['supernovae']['whichStar'][...].squeeze() ==2
survived     = f['supernovae']['Survived'][...].squeeze() == True
BH           = f['supernovae']['RemnantType'][...].squeeze() == 14
seeds        = np.in1d(f['supernovae']['randomSeed'][...].squeeze(), seedsRemain)
maskInterest = BH & secondary & survived & seeds
seedsRemain  = f['supernovae']['randomSeed'][...].squeeze()[maskInterest]
print "percentage surviving second SN = %s"  %((len(seedsRemain)/normalisation)*100)
del secondary
del BH
del survived
gc.collect()

percentage surviving second SN = 1.01536666667


0

In [35]:
seeds        = np.in1d(f['doubleCompactObjects']['seed'][...].squeeze(), seedsRemain)
Hubble       = f['doubleCompactObjects']['mergesInHubbleTimeFlag'][...].squeeze() == True
#to double check R1 RLOF1 method before I double check effect of CEE2 mask (there is none so good :) )
CEE2         = f['doubleCompactObjects']['RLOFSecondaryAfterCEE'][...].squeeze() == False
maskInterest = seeds & Hubble & CEE2
seedsRemain  = f['doubleCompactObjects']['seed'][...].squeeze()[maskInterest]
print np.sum(maskInterest)
print "percentage merging within Hubble time = %s"  %((len(seedsRemain)/normalisation)*100)

del CEE2
del Hubble 
gc.collect()

4552
percentage merging within Hubble time = 0.151733333333


0

# Double Core channel

In [65]:
# percentage of systems post MS primaries
# MS transfering onto MS secondary  (channel 1 and 2)
primaryPostHG = (f['RLOF']['type1Prev'][...].squeeze() >2)[RLOF_Z_MASK]
secondaryPostHG = (f['RLOF']['type2Prev'][...].squeeze() >2)[RLOF_Z_MASK]
firstMoment = (nthMoment == 1)[RLOF_Z_MASK]

maskInterest      = [primaryPostHG & secondaryPostHG & firstMoment]
seedsRemain       = f['RLOF']['randomSeed'][...].squeeze()[RLOF_Z_MASK][maskInterest]
nrSystemsInterest = len(seedsRemain)

testMoment   = CV.getXmomentOfMT(seedsRemain, maxCounter=10)
print np.max(testMoment)


print "post MS primar onto MS secondary "
print "of interacting systems %s percentage"\
      %((nrSystemsInterest/normalisation)*100)
del primaryPostHG
del secondaryPostHG
del firstMoment
del maskInterest
gc.collect()


1.0
post MS primar onto MS secondary 
of interacting systems 1.39753333333 percentage


0

In [66]:
# both stars are post MS
primaryPostHG = (f['commonEnvelopes']['type1'][...].squeeze() >2) &\
                (f['commonEnvelopes']['type1'][...].squeeze() <7)
secondaryPostHG = (f['commonEnvelopes']['type2'][...].squeeze()>2) &\
                (f['commonEnvelopes']['type2'][...].squeeze() <7)
seeds        = np.in1d(f['commonEnvelopes']['randomSeed'][...].squeeze(), seedsRemain)
maskInterest = primaryPostHG & secondaryPostHG & seeds 
seedsRemain    = f['commonEnvelopes']['randomSeed'][...].squeeze()[maskInterest]
print "percentage unstable-> CEE = %s"  %((len(seedsRemain)/normalisation)*100)
del primaryPostHG
del secondaryPostHG
del seeds
del maskInterest
gc.collect()

percentage unstable-> CEE = 1.26673333333


0

In [67]:
survived        = (f['commonEnvelopes']['stellarMerger'][...].squeeze() == False)
#we do the pessimistic assumption
pessimistic     = f['commonEnvelopes']['optimisticCommonEnvelopeFlag'][...].squeeze() ==  False
#they should also not RLOF after stripping directly
R1              = f['commonEnvelopes']['radius1AfterStripping'][...].squeeze()
RLOF1           = f['commonEnvelopes']['rocheLobe1to2PostCEE'][...].squeeze()
R2              = f['commonEnvelopes']['radius2AfterStripping'][...].squeeze()
RLOF2           = f['commonEnvelopes']['rocheLobe2to1PostCEE'][...].squeeze()
noRLOF          = (np.divide(R1, RLOF1) < 1) & (np.divide(R2, RLOF2) < 1)

seeds           = np.in1d(f['commonEnvelopes']['randomSeed'][...].squeeze(), seedsRemain)
maskInterest    = seeds &  survived & pessimistic & noRLOF
seedsRemain        = f['commonEnvelopes']['randomSeed'][...].squeeze()[maskInterest]
print "percentage survived =  %s"  %((len(seedsRemain)/normalisation)*100)
del survived
gc.collect()

/home/cneijssel/.local/lib/python2.7/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in less


percentage survived =  0.7143


0

In [68]:
# nr systems that survived the first SN
primary   = f['supernovae']['whichStar'][...].squeeze() ==1
survived  = f['supernovae']['Survived'][...].squeeze() == True
BH        = f['supernovae']['RemnantType'][...].squeeze() == 14
maskInterest = BH & primary & survived
print np.sum(maskInterest)
seedsSN   = f['supernovae']['randomSeed'][...].squeeze()[maskInterest]
seedsRemain = seedsRemain[(np.in1d(seedsRemain, seedsSN))]
print "percentage surviving first SN = %s"  %((len(seedsRemain)/normalisation)*100)
del primary
del BH
del survived
del seedsSN
gc.collect()

# nr systems that survived the second SN
secondary    = f['supernovae']['whichStar'][...].squeeze() ==2
survived     = f['supernovae']['Survived'][...].squeeze() == True
BH           = f['supernovae']['RemnantType'][...].squeeze() == 14
seeds        = np.in1d(f['supernovae']['randomSeed'][...].squeeze(), seedsRemain)
maskInterest = BH & secondary & survived & seeds
seedsRemain  = f['supernovae']['randomSeed'][...].squeeze()[maskInterest]
print "percentage surviving second SN = %s"  %((len(seedsRemain)/normalisation)*100)
del secondary
del BH
del survived
gc.collect()

4320585
percentage surviving first SN = 0.0922
percentage surviving second SN = 0.0449333333333


0

In [69]:
seeds        = np.in1d(f['doubleCompactObjects']['seed'][...].squeeze(), seedsRemain)
Hubble       = f['doubleCompactObjects']['mergesInHubbleTimeFlag'][...].squeeze() == True
#to double check R1 RLOF1 method before I double check effect of CEE2 mask (there is none so good :) )
CEE2         = f['doubleCompactObjects']['RLOFSecondaryAfterCEE'][...].squeeze() == False
maskInterest = seeds & Hubble & CEE2
seedsRemain  = f['doubleCompactObjects']['seed'][...].squeeze()[maskInterest]
print np.sum(maskInterest)
print "percentage merging within Hubble time = %s"  %((len(seedsRemain)/normalisation)*100)

del CEE2
del Hubble 
gc.collect()

921
percentage merging within Hubble time = 0.0307


0